In [52]:
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import sys
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
from datetime import datetime
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

In [53]:
# df = pd.read_csv('/Users/jandh/Downloads/UChicago_ProjLab2024Fall/2603_md_202106_202106.csv.gz',compression='gzip')
df = pd.read_csv('/Users/jandh/Downloads/UChicago_ProjLab2024Fall/2603_md_202106_202106.csv')

In [54]:
sys.getsizeof(df)/1000000

1876.167674

In [55]:
df.head()

,Unnamed: 0,date,time,lastPx,size,volume,SP5,SP4,SP3,SP2,...,SV5,SV4,SV3,SV2,SV1,BV1,BV2,BV3,BV4,BV5
0,0,2021-06-01,90000112,"10,150.0000","8,111.0000",8111,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3294,61,1240,1403,2195,685
1,1,2021-06-01,90000139,"10,150.0000",2.0000,8113,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3294,59,1240,1403,2195,685
2,2,2021-06-01,90000156,NaN,NaN,8113,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,59,1240,1403,2195,685
3,3,2021-06-01,90000195,"10,150.0000",16.0000,8129,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,43,1240,1403,2195,685
4,4,2021-06-01,90000197,NaN,NaN,8129,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,43,1240,1403,2195,687


In [56]:
df= df[df["SV1"]!=0]
df.head()

,Unnamed: 0,date,time,lastPx,size,volume,SP5,SP4,SP3,SP2,...,SV5,SV4,SV3,SV2,SV1,BV1,BV2,BV3,BV4,BV5
0,0,2021-06-01,90000112,"10,150.0000","8,111.0000",8111,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3294,61,1240,1403,2195,685
1,1,2021-06-01,90000139,"10,150.0000",2.0000,8113,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3294,59,1240,1403,2195,685
2,2,2021-06-01,90000156,NaN,NaN,8113,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,59,1240,1403,2195,685
3,3,2021-06-01,90000195,"10,150.0000",16.0000,8129,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,43,1240,1403,2195,685
4,4,2021-06-01,90000197,NaN,NaN,8129,"10,400.0000","10,350.0000","10,300.0000","10,250.0000",...,2596,2138,3740,2185,3324,43,1240,1403,2195,687


In [57]:
df.isna().any()

Unnamed: 0    False
date          False
time          False
lastPx         True
size           True
volume        False
SP5           False
SP4           False
SP3           False
SP2           False
SP1           False
BP1           False
BP2           False
BP3           False
BP4           False
BP5           False
SV5           False
SV4           False
SV3           False
SV2           False
SV1           False
BV1           False
BV2           False
BV3           False
BV4           False
BV5           False
dtype: bool

In [58]:
def prep_data_quantile(test_data,norm,q=0.995):
    '''quantile based threshold for labelling 999%'''
    test_data = test_data.astype(np.float64)
    for i in range(0,40,2):
        if i%2==0:
            test_data.loc[:,i] = test_data.loc[:,i]/10000
    # print(test_data.head())
    
    #Creating labels of the data
    mid_Price = (test_data.iloc[:,0]+test_data.iloc[:,2])/2
    foward_mean = mid_Price[::-1].rolling(window = 10,min_periods = 10).mean()[::-1].shift(-1)
    pc_mid = (foward_mean-mid_Price)/mid_Price

    up = pc_mid.quantile(q)
    down = pc_mid.quantile(1-q)
    print(f"upvalue= {up:.4%} , downvalue = {down:.4%} ")
    labels = pc_mid.copy(deep=True)
    labels.loc[(pc_mid<up) & (pc_mid>down)] = 1
    labels.loc[pc_mid>=up] = 0
    labels.loc[pc_mid<=down] = 2

    print(labels.unique(),labels.value_counts(),'labels distribution')
    #Normalizing using z-score
    if norm=='Z':
        scaler = StandardScaler()
        scaler.fit(test_data)
        test_data = scaler.transform(test_data)

    #Normalizing using DecPrec
    if norm=='DecPrec':
        k_len = np.ceil(np.log10(test_data.abs().max()))
        # print(k_len)
        test_data = test_data/(10**k_len)
    # test
    # print(pc_mid.iloc[-10],pc_mid.iloc[-11])
    return test_data[:-10].to_numpy(),labels[:-10].to_numpy()

In [59]:
df.columns

Index(['Unnamed: 0', 'date', 'time', 'lastPx', 'size', 'volume', 'SP5', 'SP4',
       'SP3', 'SP2', 'SP1', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'SV5', 'SV4',
       'SV3', 'SV2', 'SV1', 'BV1', 'BV2', 'BV3', 'BV4', 'BV5'],
      dtype='object')

In [60]:
def get_forward_rets(df,period = 10):
    mid_Price = (df.loc[:,'SP1']+df.loc[:,'BP1'])/2
    mid_Price = mid_Price[mid_Price!=0]
    foward_mean = mid_Price[::-1].rolling(window = period,min_periods = period).mean()[::-1].shift(-1)
    pc_mid = (foward_mean-mid_Price)/mid_Price
    pc_mid = pc_mid[pc_mid.notnull()]
    return pc_mid

In [61]:
# mid_Price = (df.loc[:,'SP1']+df.loc[:,'BP1'])/2

# # mid_Price.isna().any()
# foward_mean = mid_Price[::-1].rolling(window = 10,min_periods = 10).mean()[::-1].shift(-1)
# pc_mid = (foward_mean-mid_Price)/mid_Price
# pc_mid = pc_mid[pc_mid.notnull()]

# q = [0.0001,0.001,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,0.999,0.9999]
# for i in q:
#     # print(pc_mid[pc_mid.notnull()].quantile(q))
#     print(f"qunatile= {i:.2%} , quantileValue = {pc_mid.quantile(i):.4%}")

In [62]:
# pd.options.display.float_format = "{:,.4f}".format

In [63]:
def quantile_stats(col,suffix=None,verbose=True):
    q = [0.0001,0.001,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,0.999,0.9999]
    # ret_df = pd.DataFrame({'quantiles':q})
    # ret_df.set_index('quantiles',inplace=True)
    # ret_df['value'] = 0
    # ret_df['count']
    qvs= []
    qcs= []
    for i in q:
        
        if i>=0.05:
            quantile_count = (1-i)*len(col)
        else:
            quantile_count = i*len(col)

        qvs += [col.quantile(i)]
        qcs += [int(quantile_count)]
        if verbose:
            print(f"qunatile= {i:.2%} , quantileValue = {col.quantile(i):.4%} ,  quantile_count = {int(quantile_count)}")
    ret_df = pd.DataFrame({'quantiles':q,'values':qvs,'counts':qcs})
    ret_df.set_index(['quantiles','counts'],inplace=True)
    if suffix:
        ret_df.columns = ret_df.columns.map(lambda x: str(x) + suffix)
    return ret_df
        

In [64]:
# pd.options.display.float_format = "{:,.4f}".format
forward_10 = quantile_stats(get_forward_rets(df,10),'_fw_10')
# forward_10

qunatile= 0.01% , quantileValue = -0.4376% ,  quantile_count = 721
qunatile= 0.10% , quantileValue = -0.2685% ,  quantile_count = 7215
qunatile= 1.00% , quantileValue = 0.0000% ,  quantile_count = 72158
qunatile= 5.00% , quantileValue = 0.0000% ,  quantile_count = 6855086
qunatile= 10.00% , quantileValue = 0.0000% ,  quantile_count = 6494292
qunatile= 25.00% , quantileValue = 0.0000% ,  quantile_count = 5411910
qunatile= 50.00% , quantileValue = 0.0000% ,  quantile_count = 3607940
qunatile= 75.00% , quantileValue = 0.0000% ,  quantile_count = 1803970
qunatile= 90.00% , quantileValue = 0.0000% ,  quantile_count = 721588
qunatile= 95.00% , quantileValue = 0.0000% ,  quantile_count = 360794
qunatile= 99.00% , quantileValue = 0.0000% ,  quantile_count = 72158
qunatile= 99.90% , quantileValue = 0.2709% ,  quantile_count = 7215
qunatile= 99.99% , quantileValue = 0.4454% ,  quantile_count = 721


In [65]:
spread = (df.loc[:,'SP1']-df.loc[:,'BP1'])/((df.loc[:,'SP1']+df.loc[:,'BP1'])/2)
# spread.head()
spread_quantiles = quantile_stats(spread,'_spread')
# spread_quantiles

qunatile= 0.01% , quantileValue = 0.1001% ,  quantile_count = 722
qunatile= 0.10% , quantileValue = 0.1001% ,  quantile_count = 7221
qunatile= 1.00% , quantileValue = 0.3407% ,  quantile_count = 72219
qunatile= 5.00% , quantileValue = 0.3454% ,  quantile_count = 6860824
qunatile= 10.00% , quantileValue = 0.3527% ,  quantile_count = 6499728
qunatile= 25.00% , quantileValue = 0.3697% ,  quantile_count = 5416440
qunatile= 50.00% , quantileValue = 0.4057% ,  quantile_count = 3610960
qunatile= 75.00% , quantileValue = 0.4357% ,  quantile_count = 1805480
qunatile= 90.00% , quantileValue = 0.4494% ,  quantile_count = 722192
qunatile= 95.00% , quantileValue = 0.4684% ,  quantile_count = 361096
qunatile= 99.00% , quantileValue = 0.4938% ,  quantile_count = 72219
qunatile= 99.90% , quantileValue = 0.8333% ,  quantile_count = 7221
qunatile= 99.99% , quantileValue = 0.9390% ,  quantile_count = 722


In [66]:
forward = spread_quantiles.merge(forward_10,left_on=['quantiles'],right_on=['quantiles'])
ranges = [20,50,100,200,500]
for i in ranges:
    temp = quantile_stats(get_forward_rets(df,i),'_fw_%d'%i,False)
    forward = forward.merge(temp,left_on=['quantiles'],right_on=['quantiles'])


Thresh_holding Summary

In [67]:
pd.options.display.float_format = "{:,.4f}".format
forward

,values_spread,values_fw_10,values_fw_20,values_fw_50,values_fw_100,values_fw_200,values_fw_500
quantiles,,,,,,,
0.0001,0.0010,-0.0044,-0.0044,-0.0045,-0.0048,-0.0064,-0.0101
0.0010,0.0010,-0.0027,-0.0033,-0.0038,-0.0040,-0.0042,-0.0044
0.0100,0.0034,0.0000,0.0000,-0.0010,-0.0020,-0.0029,-0.0035
0.0500,0.0035,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0015
0.1000,0.0035,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.2500,0.0037,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.5000,0.0041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.7500,0.0044,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.9000,0.0045,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [68]:
forward['count'] = forward.index.map(lambda x: int(x*len(df)) if x<=0.5 else int((1-x)*len(df)))
stats_df = forward.reset_index()
stats_df.set_index(['quantiles','count'],inplace=True)
stats_df.style.format('{:,.2%}')

,,values_spread,values_fw_10,values_fw_20,values_fw_50,values_fw_100,values_fw_200,values_fw_500
quantiles,count,,,,,,,
0.000100,722,0.10%,-0.44%,-0.44%,-0.45%,-0.48%,-0.64%,-1.01%
0.001000,7221,0.10%,-0.27%,-0.33%,-0.38%,-0.40%,-0.42%,-0.44%
0.010000,72219,0.34%,0.00%,0.00%,-0.10%,-0.20%,-0.29%,-0.35%
0.050000,361096,0.35%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.15%
0.100000,722192,0.35%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
0.250000,1805480,0.37%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
0.500000,3610960,0.41%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
0.750000,1805480,0.44%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
0.900000,722192,0.45%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%


In [69]:
# looks like from 9-13 5 hrs per day?
df['time'].describe()

count     7,221,921.0000
mean    106,687,343.7075
std      13,803,992.6815
min      90,000,002.0000
25%      92,924,107.0000
50%     103,920,489.0000
75%     120,619,752.0000
max     132,459,996.0000
Name: time, dtype: float64

In [98]:
#confirming HHmmSS.sss
df[df['time'] == 132459996]
max((df['time']//1000000)%10)
# 90000009 - 132459996

5

In [71]:
diff = pd.to_numeric(df['time']).diff()
diff.describe()

count     7,221,920.0000
mean              5.8165
std          67,785.7936
min     -42,459,987.0000
25%               1.0000
50%               9.0000
75%              30.0000
max       4,500,175.0000
Name: time, dtype: float64

In [90]:
# issue with things with same time but a small trade occcurs
# df[df['time'].diff()==0].head()

In [50]:
# df.loc[292,:] == df.loc[293,:]
# df.loc[292,:]

In [78]:
# removing outliers
# pd.options.display.float_format = "{:,.2f}".format
# diff = abs(diff[abs(diff)<50000])
for i in [0.0001,0.001,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,0.999,0.9999]:
    print(f"qunatile= {i:.2%} , quantileValue = {abs(diff).quantile(i):.6}")

qunatile= 0.01% , quantileValue = 0.0
qunatile= 0.10% , quantileValue = 0.0
qunatile= 1.00% , quantileValue = 0.0
qunatile= 5.00% , quantileValue = 0.0
qunatile= 10.00% , quantileValue = 0.0
qunatile= 25.00% , quantileValue = 1.0
qunatile= 50.00% , quantileValue = 9.0
qunatile= 75.00% , quantileValue = 30.0
qunatile= 90.00% , quantileValue = 71.0
qunatile= 95.00% , quantileValue = 110.0
qunatile= 99.00% , quantileValue = 236.0
qunatile= 99.90% , quantileValue = 549.0
qunatile= 99.99% , quantileValue = 40280.0


## avg tick time 10 ms

In [80]:
#1000000 = 3600 Sec
for i in [10,20,50,100,200,500]:
    print(f"{10*i/(10**3)}  seconds for forward look {i} ticks")

0.1  seconds for forward look 10 ticks
0.2  seconds for forward look 20 ticks
0.5  seconds for forward look 50 ticks
1.0  seconds for forward look 100 ticks
2.0  seconds for forward look 200 ticks
5.0  seconds for forward look 500 ticks


Points of uptrends,lowtrends

In [81]:
pc_mid = get_forward_rets(df,period = 10)


In [82]:
analysis_df = pd.DataFrame({'fw_10':pc_mid})
analysis_df = pd.merge(analysis_df, df[['date','time']],  how='left',left_index=True, right_index=True)

In [83]:
analysis_df = analysis_df[['date','time','fw_10']]
analysis_df['up'] = 0
analysis_df['down'] = 0
analysis_df.loc[analysis_df['fw_10']>=analysis_df['fw_10'].quantile(0.999),'up'] = 1
analysis_df.loc[analysis_df['fw_10']<=analysis_df['fw_10'].quantile(0.001),'down'] = 1
analysis_df.head()

,date,time,fw_10,up,down
0,2021-06-01,90000112,0.0000,0,0
1,2021-06-01,90000139,0.0000,0,0
2,2021-06-01,90000156,0.0000,0,0
3,2021-06-01,90000195,0.0000,0,0
4,2021-06-01,90000197,0.0000,0,0


In [84]:
analysis_df['date_delta_10'] = pd.to_datetime(analysis_df['date']) - pd.to_datetime(analysis_df['date'].shift(-11))
analysis_df['time_delta_10'] = pd.to_numeric(analysis_df['time']) - pd.to_numeric(analysis_df['time']).shift(-11)
analysis_df.head()

,date,time,fw_10,up,down,date_delta_10,time_delta_10
0,2021-06-01,90000112,0.0000,0,0,0 days,-352.0000
1,2021-06-01,90000139,0.0000,0,0,0 days,-345.0000
2,2021-06-01,90000156,0.0000,0,0,0 days,-363.0000
3,2021-06-01,90000195,0.0000,0,0,0 days,-382.0000
4,2021-06-01,90000197,0.0000,0,0,0 days,-398.0000


In [85]:
up_ones = analysis_df[analysis_df['up']==1]
down_ones = analysis_df[analysis_df['down']==1]

In [86]:
up_ones[['time_delta_10']].describe()

,time_delta_10
count,"7,230.0000"
mean,"519,105.6533"
std,"5,345,311.7097"
min,"-7,000,828.0000"
25%,-216.0000
50%,-119.0000
75%,-61.0000
max,"42,459,898.0000"


In [87]:
up_ones['date_delta_10'].value_counts()

date_delta_10
0 days     7099
-1 days      98
-3 days      24
-4 days       9
Name: count, dtype: int64

In [88]:
# analysis_df['datetime'] = pd.to_datetime(analysis_df['date']) + analysis_df['time'].apply(lambda x: pd.Timedelta(x, unit='ns'))

In [93]:
temp  = pd.to_datetime(analysis_df['date'])
print(temp.iloc[0] - temp.iloc[-1])
len(df)/28

-28 days +00:00:00


257925.75

In [97]:
# 257925*0.01 

## Try 10_forward, 50_forward, 100_forward for labelling